In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/IBM - Data Science/Analysis project(IBM-Nation)/Project-1-Analysis/Files/Product Data Set - Student 2 of 3.csv"
product_data = pd.read_csv(path, sep = '|')
product_data.head(2)

,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,30001,HEALTH & BEAUTY,$7.45
1,30002,HEALTH & BEAUTY,$5.35


In [ ]:
path = "/content/drive/MyDrive/IBM - Data Science/Analysis project(IBM-Nation)/Project-1-Analysis/Files/Transaction Data Set - Student 3 of 3.csv"
transactions_data = pd.read_csv(path, sep = '|')
transactions_data.head(2)

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT
0,10114,30011,4,0.0,1/2/2015,0
1,10217,30016,3,0.0,1/2/2015,0


In [ ]:
path = "/content/drive/MyDrive/IBM - Data Science/Analysis project(IBM-Nation)/Project-1-Analysis/Files/Customer Data Set - Student 1 of 3.csv"
customer_data = pd.read_csv(path, sep = ',')
customer_data.head(2)

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS
0,10001,0,64,"$133,498",5,enrolled,06-03-2013,4,Single
1,10002,0,42,"$94,475",9,notenrolled,NaN,6,Married


In [ ]:
customer_data['INCOME'] = customer_data['INCOME'].map(lambda x: x.replace('$','')) 
customer_data['INCOME'] = customer_data['INCOME'].map(lambda x: int(x.replace(',',''))) 

In [ ]:
product_data['UNIT LIST PRICE'] = product_data['UNIT LIST PRICE'].map(lambda x: float(x.replace('$',''))) 

In [ ]:
trans_products = transactions_data.merge(product_data, how= 'inner', left_on ='PRODUCT NUM', right_on = 'PRODUCT CODE')
trans_products.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,10114,30011,4,0.0,1/2/2015,0,30011,APPAREL,25.46
1,10086,30011,6,0.0,1/2/2015,0,30011,APPAREL,25.46
2,10174,30011,10,0.0,1/2/2015,0,30011,APPAREL,25.46
3,10401,30011,12,0.0,1/2/2015,0,30011,APPAREL,25.46
4,10216,30011,12,0.1,1/2/2015,0,30011,APPAREL,25.46


In [ ]:
trans_products['Total_Price'] = trans_products['QUANTITY PURCHASED'] * trans_products['UNIT LIST PRICE'] * (1-trans_products['DISCOUNT TAKEN'])
trans_products.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE,Total_Price
0,10114,30011,4,0.0,1/2/2015,0,30011,APPAREL,25.46,101.840
1,10086,30011,6,0.0,1/2/2015,0,30011,APPAREL,25.46,152.760
2,10174,30011,10,0.0,1/2/2015,0,30011,APPAREL,25.46,254.600
3,10401,30011,12,0.0,1/2/2015,0,30011,APPAREL,25.46,305.520
4,10216,30011,12,0.1,1/2/2015,0,30011,APPAREL,25.46,274.968


In [ ]:
customer_prod_categ = trans_products.groupby(['CUSTOMER NUM','PRODUCT CATEGORY']).agg({'Total_Price': 'sum'})
customer_prod_categ.head()

Total_Price
CUSTOMER NUM PRODUCT CATEGORY             
10001        APPAREL              4022.430
             ELECTRONICS          1601.315
             FOOD                   68.688
             HEALTH & BEAUTY      1134.337
10002        APPAREL              2312.509

In [ ]:
customer_prod_categ.reset_index()

,CUSTOMER NUM,PRODUCT CATEGORY,Total_Price
0,10001,APPAREL,4022.430
1,10001,ELECTRONICS,1601.315
2,10001,FOOD,68.688
3,10001,HEALTH & BEAUTY,1134.337
4,10002,APPAREL,2312.509
...,...,...,...
1422,10498,HEALTH & BEAUTY,320.145
1423,10499,HEALTH & BEAUTY,20.010
1424,10500,APPAREL,1511.748
1425,10500,ELECTRONICS,2632.269


In [ ]:
customer_pivot = customer_prod_categ.pivot_table(index='CUSTOMER NUM', columns='PRODUCT CATEGORY', values='Total_Price')
customer_pivot.head()

PRODUCT CATEGORY,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY
CUSTOMER NUM,,,,
10001,4022.430,1601.315,68.688,1134.337
10002,2312.509,2473.163,276.779,NaN
10003,2887.382,5414.418,260.640,NaN
10004,3637.213,1840.211,45.270,NaN
10005,213.512,NaN,NaN,NaN


In [ ]:
trans_total_spend = trans_products.groupby('CUSTOMER NUM').agg({'Total_Price': 'sum'}).\
rename(columns = {'Total_Price':'Total Spent'})
trans_total_spend.head()

,Total Spent
CUSTOMER NUM,
10001,6826.770
10002,5062.451
10003,8562.440
10004,5522.694
10005,213.512


In [ ]:
customer_KPIS = customer_pivot.merge(trans_total_spend, how = 'inner', left_index= True , right_index = True)
customer_KPIS.head()

,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,Total Spent
CUSTOMER NUM,,,,,
10001,4022.430,1601.315,68.688,1134.337,6826.770
10002,2312.509,2473.163,276.779,NaN,5062.451
10003,2887.382,5414.418,260.640,NaN,8562.440
10004,3637.213,1840.211,45.270,NaN,5522.694
10005,213.512,NaN,NaN,NaN,213.512


In [ ]:
customer_all_view = customer_data.merge(customer_KPIS, how ='inner', left_on='CUSTOMERID', right_index=True)
customer_all_view=customer_all_view.fillna(0)
customer_all_view.head()

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,Total Spent
0,10001,0,64,133498,5,enrolled,06-03-2013,4,Single,4022.430,1601.315,68.688,1134.337,6826.770
1,10002,0,42,94475,9,notenrolled,0,6,Married,2312.509,2473.163,276.779,0.000,5062.451
2,10003,0,40,88610,9,enrolled,02-09-2010,5,Married,2887.382,5414.418,260.640,0.000,8562.440
3,10004,0,38,84313,8,enrolled,06-04-2015,1,Single,3637.213,1840.211,45.270,0.000,5522.694
4,10005,0,30,51498,3,notenrolled,0,1,Single,213.512,0.000,0.000,0.000,213.512


# Regression

In [ ]:
DF_input=customer_all_view[['GENDER','AGE','INCOME','EXPERIENCE SCORE','LOYALTY GROUP','HOUSEHOLD SIZE','MARITAL STATUS']]
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,enrolled,4,Single
1,0,42,94475,9,notenrolled,6,Married
2,0,40,88610,9,enrolled,5,Married
3,0,38,84313,8,enrolled,1,Single
4,0,30,51498,3,notenrolled,1,Single


In [ ]:
# Loyalty group and Marital status are strings
DF_input["MARITAL STATUS"].unique()

array(['Single', 'Married', 'Divorced', 'Widow/Widower'], dtype=object)

In [ ]:
DF_input["LOYALTY GROUP"].unique()

array(['enrolled', 'notenrolled'], dtype=object)

In [ ]:
#  pd.get_dummies(DF_input["MARITAL STATUS"]).head()

In [ ]:
#  pd.get_dummies(DF_input["LOYALTY GROUP"]).head()

In [ ]:
def encode_loyalty(value):
  if value =='enrolled':
    return 1
  else:
   return 0

In [ ]:
DF_input["LOYALTY GROUP"] = DF_input["LOYALTY GROUP"].apply(encode_loyalty)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# DF_input.head()

If we run line 24 firstly it will divide LOYALTY GROUP column into two cols, and we don't need that. So, we make a function for this col separately

In [ ]:
DF_input = pd.get_dummies(DF_input)
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0,64,133498,5,1,4,0,0,1,0
1,0,42,94475,9,0,6,0,1,0,0
2,0,40,88610,9,1,5,0,1,0,0
3,0,38,84313,8,1,1,0,0,1,0
4,0,30,51498,3,0,1,0,0,1,0


In [ ]:
from sklearn import preprocessing

In [ ]:
DF_input_column_names = DF_input.columns.values

In [ ]:
DF_input_np=preprocessing.minmax_scale(DF_input)

In [ ]:
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)
Reg_input_scaled.head()


,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0.0,0.638889,0.872818,0.444444,1.0,0.6,0.0,0.0,1.0,0.0
1,0.0,0.333333,0.572046,0.888889,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.305556,0.526842,0.888889,1.0,0.8,0.0,1.0,0.0,0.0
3,0.0,0.277778,0.493722,0.777778,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.166667,0.240799,0.222222,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(Reg_input_scaled,customer_all_view['Total Spent']  ,test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
regr = LinearRegression()
regr.fit(X_train,Y_train)
Y_pred = regr.predict(X_test)
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [   35.58494877 -7076.18001534 12403.0952701    505.84578936
  1566.48092344  1040.19923196   377.33573376   -82.02665324
   -34.57620088  -260.73287964]


In [ ]:
import numpy as np
np.positive(regr.coef_)

array([   35.58494877, -7076.18001534, 12403.0952701 ,   505.84578936,
        1566.48092344,  1040.19923196,   377.33573376,   -82.02665324,
         -34.57620088,  -260.73287964])

In [ ]:
Reg_input_scaled.columns

Index(['GENDER', 'AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP',
       'HOUSEHOLD SIZE', 'MARITAL STATUS_Divorced', 'MARITAL STATUS_Married',
       'MARITAL STATUS_Single', 'MARITAL STATUS_Widow/Widower'],
      dtype='object')

In [ ]:
coeficient_ = pd.DataFrame( regr.coef_ ,columns=['Coefficient'])
coeficient_.head()

,Coefficient
0,35.584949
1,-7076.180015
2,12403.095270
3,505.845789
4,1566.480923


In [ ]:
coef_list = []

for num in regr.coef_:
  if num > 0:
    print("Positive")
  else:
    print("Negative")
  coef_list.append(num)
print(coef_list)
  

Positive
Negative
Positive
Positive
Positive
Positive
Positive
Negative
Negative
Negative
[35.58494877071031, -7076.180015341912, 12403.095270099186, 505.84578936217963, 1566.4809234351171, 1040.1992319562994, 377.3357337621476, -82.0266532401771, -34.57620088164231, -260.73287964032716]


In [ ]:
import pandas as pd
DF_input_column_names = DF_input.columns

predictor_varibale_table = pd.DataFrame(data = DF_input_column_names)
predictor_varibale_table.columns = ['predictor varibale']
# predictor_varibale_table.head(10)

table = pd.concat([predictor_varibale_table,coeficient_],axis=1)
table.head(10)


,predictor varibale,Coefficient
0,GENDER,35.584949
1,AGE,-7076.180015
2,INCOME,12403.095270
3,EXPERIENCE SCORE,505.845789
4,LOYALTY GROUP,1566.480923
5,HOUSEHOLD SIZE,1040.199232
6,MARITAL STATUS_Divorced,377.335734
7,MARITAL STATUS_Married,-82.026653
8,MARITAL STATUS_Single,-34.576201
9,MARITAL STATUS_Widow/Widower,-260.732880


In [ ]:
Relation_Total_Spend = pd.DataFrame( coef_list,columns= ['Relationship with Total Spend'])
Relation_Total_Spend.head()

,Relationship with Total Spend
0,35.584949
1,-7076.180015
2,12403.095270
3,505.845789
4,1566.480923


In [ ]:
table_last = pd.concat([table,Relation_Total_Spend],axis=1)
table_last.head(10)

,predictor varibale,Coefficient,Relationship with Total Spend
0,GENDER,35.584949,35.584949
1,AGE,-7076.180015,-7076.180015
2,INCOME,12403.095270,12403.095270
3,EXPERIENCE SCORE,505.845789,505.845789
4,LOYALTY GROUP,1566.480923,1566.480923
5,HOUSEHOLD SIZE,1040.199232,1040.199232
6,MARITAL STATUS_Divorced,377.335734,377.335734
7,MARITAL STATUS_Married,-82.026653,-82.026653
8,MARITAL STATUS_Single,-34.576201,-34.576201
9,MARITAL STATUS_Widow/Widower,-260.732880,-260.732880
